In [1]:
import pandas as pd
import numpy as np
import os
from laspy.file import File
from os import listdir
from os.path import isfile, join
from pathlib import Path
import pptk
import matplotlib.pyplot as plt

# Dataframe Creation

In [2]:
def makeDf(file_location_las):
    """
    file_location_las: Path to the las file.
    return:
        pandas data frame with features
            x ,y ,z ( in meters )
            r ,g ,b
    """
    inFile = File(file_location_las, mode='r')
    df_las = pd.DataFrame(columns = ["X", "Y", "Z", "Red", "Green", "Blue"])
    df_las['X'] = inFile.get_x_scaled()
    df_las['Y'] = inFile.get_y_scaled()
    df_las['Z'] = inFile.get_z_scaled()
    df_las['Green'] = inFile.green/256
    df_las['Red'] = inFile.red/256
    df_las['Blue'] = inFile.blue/256

    return df_las

def makeDf_aerial(file_location_las):
    """
    file_location_las: Path to the las file.
    return:
        pandas data frame with features
            x ,y ,z ( in meters )
            r ,g ,b
    """
    inFile = File(file_location_las, mode='r')
    df_las = pd.DataFrame(columns = ["X", "Y", "Z"])
    df_las['X'] = inFile.get_x_scaled()
    df_las['Y'] = inFile.get_y_scaled()
    df_las['Z'] = inFile.get_z_scaled()
    df_las['Classification'] = inFile.Classification
    df_las['Intensity'] = inFile.Intensity

    return df_las

In [ ]:
file_root_path = Path(r"C:\Users\Kazi Abir Adnan\The University of Melbourne")
las_format_folder = file_root_path / 'Yuming ZHANG - data'
mypath = r"C:\Users\Kazi Abir Adnan\The University of Melbourne\Yuming ZHANG - data"
files = [f for f in listdir(mypath) if isfile(join(mypath, f)) and f.endswith('las')]

In [ ]:
data = []
for index, file in enumerate(files):
    if index != 21 and index != 22:
        file_location = las_format_folder/file
        df = makeDf(file_location)
        D = 10
        df_filtered_10 = df[(df['X'] <= D) & (df['X'] >= -D) & (df['Y'] <= D) & (df['Y'] >= -D)]
        D = 15
        df_filtered_15 = df[(df['X'] <= D) & (df['X'] >= -D) & (df['Y'] <= D) & (df['Y'] >= -D)]
        D = 20
        df_filtered_20 = df[(df['X'] <= D) & (df['X'] >= -D) & (df['Y'] <= D) & (df['Y'] >= -D)]
        D = 30
        df_filtered_30 = df[(df['X'] <= D) & (df['X'] >= -D) & (df['Y'] <= D) & (df['Y'] >= -D)]
        data.append({'file_name': file, 'size' : df.shape[0], 'file_size': os.stat(file_location).st_size >> 20,
                     'x_min':df['X'].min(), 'x_max': df['X'].max(),
                     'y_min':df['Y'].min(), 'y_max': df['Y'].max(),
                     'z_min':df['Z'].min(), 'z_max': df['Z'].max(),
                     'green_mean':df['Green'].mean(), 'red_mean': df['Red'].mean(), 'Blue_mean':df['Blue'].mean(),
                     'diameter_10_size': df_filtered_10.shape[0] / df.shape[0],
                     'diameter_15_size': df_filtered_15.shape[0] / df.shape[0],
                     'diameter_20_size': df_filtered_20.shape[0] / df.shape[0],
                     'diameter_30_size': df_filtered_30.shape[0] / df.shape[0]})
        print(index, file)

In [ ]:
df = pd.DataFrame(data)
df.to_csv('dataset_overview.csv')

# Statistics of Data

#### Diameter Selection

In [ ]:
plt.plot( list(range(0,len(df))), 'diameter_10_size', data=df, marker='o', markerfacecolor='blue', markersize=8, color='skyblue', linewidth=2, label = '10X10')
plt.plot( list(range(0,len(df))), 'diameter_15_size', data=df, marker='', color='olive', linewidth=2, linestyle='--', label = '15X15')
plt.plot( list(range(0,len(df))), 'diameter_20_size', data=df, marker='', color='olive', linewidth=2, linestyle='-.', label = '20X20')
plt.plot( list(range(0,len(df))), 'diameter_30_size', data=df, marker='', color='olive', linewidth=2, linestyle=':', label = '30X30')
plt.legend()
plt.savefig('diameter_selection.png', dpi=300,transparent=True)

#### Min Max of xyz

In [ ]:
plt.plot( list(range(0,len(df))), 'x_max', data=df, marker='', color='olive', linewidth=2, linestyle='-')
plt.plot( list(range(0,len(df))), 'x_min', data=df, marker='', color='blue', linewidth=2, linestyle='-')
plt.plot( list(range(0,len(df))), 'y_max', data=df, marker='', color='olive', linewidth=2, linestyle='--')
plt.plot( list(range(0,len(df))), 'y_min', data=df, marker='', color='blue', linewidth=2, linestyle='--')
plt.plot( list(range(0,len(df))), 'z_max', data=df, marker='', color='olive', linewidth=2, linestyle=':')
plt.plot( list(range(0,len(df))), 'z_min', data=df, marker='', color='blue', linewidth=2, linestyle=':')
plt.legend()
plt.savefig('xyz_min_max.png', dpi=300,transparent=True)

#### File Size

In [ ]:
plt.subplot(1, 2, 1)
plt.plot( list(range(0,len(df))), 'file_size', data=df, marker='', color='olive', linewidth=2, linestyle='-', label = 'File Size in MB')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot( list(range(0,len(df))), 'size', data=df, marker='', color='blue', linewidth=2, linestyle='--', label = "# of points")
plt.legend()
plt.savefig('file_size.png', dpi=300,transparent=True)

#### RGB

In [ ]:
plt.plot( list(range(0,len(df))), 'red_mean', data=df, marker='', color='red', linewidth=2, linestyle='-')
plt.plot( list(range(0,len(df))), 'green_mean', data=df, marker='', color='green', linewidth=2, linestyle='-')
plt.plot( list(range(0,len(df))), 'Blue_mean', data=df, marker='', color='blue', linewidth=2, linestyle='-')
plt.legend()
plt.savefig('rgb.png', dpi=300,transparent=True)

# 3D plot

In [3]:
def plot_pptk(df):
    v = pptk.viewer(df[['X', 'Y', 'Z']])
    v.attributes(df[['Red', 'Green', 'Blue']] / 255.)
    v.set(point_size=0.001)
    return v

def plot_pptk_aerial(df):
    v = pptk.viewer(df[['X', 'Y', 'Z']])
    v.attributes(df['Classification'])
    v.set(point_size=0.001)
    return v

In [4]:
file_location = r"C:\Users\Kazi Abir Adnan\The University of Melbourne\Yuming ZHANG - data\carpark.las"
df_terrestrial = makeDf(file_location)
v = plot_pptk(df_terrestrial)

In [10]:
poses = []
poses.append([0, 0, 0, 0 * np.pi/4, np.pi/6, 50])
poses.append([0, 0, 0, 1 * np.pi/4, np.pi/6, 50])
poses.append([0, 0, 0, 2 * np.pi/4, np.pi/6, 50])
poses.append([0, 0, 0, 3 * np.pi/4, np.pi/6, 50])
poses.append([0, 0, 0, 4 * np.pi/4, np.pi/6, 50])

In [16]:
v.play(poses, 8 * np.arange(5), repeat=True, interp='linear')
# v.record('recording', poses, 8 * np.arange(5), interp='linear')

In [15]:
v.capture('aerial_y.png')

In [ ]:
file_location = r"C:\Users\Kazi Abir Adnan\OneDrive - The University of Melbourne\aerial\Area 13\717000_5952000.las"
df_arial = makeDf_aerial(file_location)
v = plot_pptk_aerial(df_arial)

In [ ]:
labels = ['Overlap/Reserved', 'High Vegetation', 'Ground', 'Low Vegetation', 'Medium Vegetation', 'Unassigned', 'Noise']
sizes = x.values
colors = ['purple', 'olive', 'coral', 'green','chartreuse','gray','red']
patches, texts = plt.pie(sizes, colors=colors)
plt.legend(patches, labels)
plt.axis('equal')
plt.tight_layout()
plt.savefig('classification_aerial.png', dpi=300,transparent=True)